# 1. 챗봇 구축을 위한 데이터 수집

---

- 커리어넷 홈페이지에서 스크래핑을 통해 "직업백과", "전공정보", "대학정보" 데이터를 수집함.
- [커리어넷 바로가기](https://www.career.go.kr/cnet/front/main/main.do)

# 1) 환경 설정

- 스크래핑을 하기 위해 Selenium 사용함.
- 크롬드라이버 경로와 스크래핑할 데이터들이 위치한 커리어넷 직업백과 페이지, 학과정보 페이지, 학교(대학)정보 페이지 URL 주소를 config에 저장함.

In [1]:
%%capture --no-stderr
%pip install -U --quiet selenium webdriver-manager

In [2]:
import re
import os
import time
import json
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException
from abc import ABC, abstractmethod

In [3]:
config = {
    "chrome_driver_path" : f'./ChromeDriver/chromedriver',
    "URL": {
        "careernet_careerinfo" : "https://www.career.go.kr/cnet/front/base/job/jobList.do#tab1",
        "careernet_majorinfo" : "https://www.career.go.kr/cnet/front/base/major/FunivMajorList.do",
        "careernet_univinfo" : "https://www.career.go.kr/cnet/front/base/school/schoolUniversityList.do",
    },
}

# 2) 스크래핑을 위한 헬퍼 클래스 선언

- `get_chrome_driver()` : 크롬 드라이버 객체를 생성
- `wait_for_element()` : 웹 페이지에 찾고자 하는 요소가 완전히 로드될 때까지 대기
- `wait_for_page()` : 웹페이지가 완전히 로드될 때까지 대기
- `close_driver()` : 드라이버 삭제 및 종료

In [4]:
class ScrapingTools:
    @staticmethod        
    def get_chrome_driver(config):
        """ 크롬 드라이버 객체를 생성하는 함수 """
        # Chrome options
        options = webdriver.ChromeOptions()
        options.add_argument('--incognito')               # 크롬 시크릿 모드로 실행
        options.add_argument("--start-maximized")         # 브라우저 창 최대화
        options.add_argument('--disable-gpu')             # GPU 기반/보조 렌더링을 비활성화
        options.add_argument('--no-sandbox')              # 보안 샌드박스 모드 비활성화 : 리눅스에서 루트 사용자로 실행할 때 필요
        options.add_argument('--disable-dev-shm-usage')   # 공유 메모리 제한 해제
        options.add_argument('--disable-cache')         # 캐시 삭제
        options.add_argument('--headless')              # headless 모드로 실행
        
        # 크롬 드라이버 자동 업데이트 및 크롬 드라이버 객체 생성
        driver_path = config['chrome_driver_path']
        try:
            service = Service(executable_path=driver_path)
            driver = webdriver.Chrome(options=options, service=service)
        except:
            if os.path.exists(driver_path):
                os.remove(driver_path)

            new_driver_path = ChromeDriverManager().install()
            service = Service(executable_path=new_driver_path)
            driver = webdriver.Chrome(options=options, service=service)

        # 웹 드라이버 자동화 탐지 방지 스크립트 실행
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """}) 
        driver.execute_cdp_cmd("Network.enable", {}) # 네트워크 활성화
        
        print('[ScrapingTools] 🟡 크롬 드라이버 객체 생성 완료')
        return driver
    
    @staticmethod    
    def wait_for_element(driver, element_type, by_type, identifier, timeout=15):
        """ 웹 페이지에 특정 요소가 완전히 로드될 때까지 대기하는 함수 """
        try:
            wait_element = WebDriverWait(driver, timeout)
            if element_type == 'locate': # 찾고자 하는 요소가 존재할 때까지 대기
                element = wait_element.until(EC.presence_of_element_located((by_type, identifier)))
                return element
            elif element_type == 'click': # 찾고자 하는 요소가 클릭가능할 때까지 대기
                element = wait_element.until(EC.element_to_be_clickable((by_type, identifier)))
                return element
        except TimeoutException:
            print(f"[ScrapingTools] 🔴 '{identifier}' 요소 대기 시간 초과: ")
            return None
        
    @staticmethod    
    def wait_for_page(driver, timeout=15):
        """ 페이지 전체가 완전히 로드될 때까지 대기하는 함수 """
        WebDriverWait(driver, timeout).until(lambda d: d.execute_script("return document.readyState === 'complete'"))
    
    @staticmethod    
    def close_driver(driver):
        """ 드라이버 종료 """
        if driver:
            driver.close()
            driver.quit()
            print('[ScrapingTools] 🟡 크롬 드라이버 종료')

# 3) 스크래핑 작업을 위한 추상 클래스 선언

- `scraping_urls()` : 게시판에서 게시물의 URL을 수집
- `scraping_data()` : 수집된 URL을 순회하며 데이터 수집
- `save_results_json()` : 수집된 데이터를 JSON으로 저장
- `save_results_markdown()` : 수집된 데이터를 마크다운으로 저장
- `close()` : 드라이버 종료
- `run_scraping()` : 스크래핑 및 저장 스크립트 일괄 실행

In [6]:
class BaseScraper(ABC):
    def __init__(self, config):
        self.driver = ScrapingTools.get_chrome_driver(config)
        self.config = config
        self.urls = [] # 스크래핑할 페이지 URL 저장
        self.data = {} # 스크래핑한 데이터 저장
    
    @abstractmethod
    def scraping_urls(self):
        """ URL 수집 추상 메서드 """
        pass
    
    @abstractmethod
    def scraping_data(self, url):
        """ 게시글 데이터 수집 추상 메서드 """
        pass
    
    def save_results_json(self, filename):
        """JSON 파일로 저장하는 기능"""
        filename = filename+'.json'
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(self.data, f, ensure_ascii=False, indent=4)
        print(f"[BaseScraper] 🟢 결과가 {filename} 파일에 JSON 형식으로 저장되었습니다.")
    
    @abstractmethod
    def save_results_markdown(self, filename):
        """마크다운 파일로 저장하는 기본 기능"""
        pass
    
    def close(self):
        ScrapingTools.close_driver(self.driver)
        
    def run_scraping(self, filename):
        """URL 수집, 데이터 수집, 결과 저장을 일괄 실행"""
        print(f"[BaseScraper] 🟢 스크래핑 작업 시작")
        self.scraping_urls() 
        self.scraping_data() 
        self.save_results_json(filename=filename)
        self.save_results_markdown(filename=filename)
        self.close()
        print(f"[BaseScraper] 🟢 스크래핑 작업 완료")
        

# 4) 커리어넷 직업백과 데이터 수집을 위한 클래스 선언

- `scraping_urls()` : 직업백과 페이지에서 개별 직업 정보가 담긴 게시물들의 URL 주소 수집
- `scraping_data()` : 직업백과 페이지에서 수집한 직업 게시물 페이지 URL들을 순회하며 실제 데이터 수집 

In [13]:
class CareerScraper(BaseScraper):     
    def scraping_urls(self):
        """ 직업 정보 페이지에서 개별 직업 정보가 담긴 페이지의 URL 주소 수집 """
        print("[CareerScraper] 🔵 직업 정보 데이터 수집 시작 (개별 직업 설명이 적힌 URL 수집)")
        start_time = time.time()
        self.driver.get(self.config["URL"]["careernet_careerinfo"]) # 커리어넷 직업백과 URL 접속
        self._click_filter()                                # 일관성 있게 수집하기 위해 직업 정보 게시판 콘텐츠 정렬
        total_counts = self._get_contents_count()                       # 전체 게시물 개수 가져오기
        
        page_count = 1  # 게시판 페이지 번호 추적
        while len(self.urls) < total_counts:  # 수집된 URL 개수가 '전체 게시물 개수'와 같을 때까지 수집
            self._get_urls()
            print(f"[CareerScraper] ⚪️ 페이지 {page_count} 스크래핑 완료. 수집한 누적 직업 데이터 개수: {len(self.urls)}")
            page_count += 1

            # 스크래핑 종료 기준
            if len(self.urls) >= total_counts:
                print("[CareerScraper] 🔵 모든 게시물을 다 스크래핑했습니다.")
                break

            # 페이징 바 클릭하여, 다음 페이지로 이동
            if not self._click_nextpage(page_count):
                break
            
        elapsed_time = time.time() - start_time
        print(f'[CareerScraper] 🔵 총 수집한 직업 정보 URL 주소 : {len(self.urls)} 개 | ⏱️ 소요 시간: {elapsed_time:.2f} 초')
        
    def scraping_data(self):
        """ 직업 정보 URL 리스트에서 직업 정보 수집 """
        print(f"[CareerScraper] 🔵 URL 리스트로 부터 데이터 수집 시작")
        start_time = time.time()
        total_urls = len(self.urls)  # 전체 URL 개수
        
        for idx, url in enumerate(self.urls, start=1):
            print(f"[CareerScraper] 🔵 수집 타겟 URL 👉🏻 {url}")

            self.driver.get(url)  # 직업 정보 페이지 접근
            job_details = self._get_details()
            self.data[job_details["직업명"]] = job_details  # 직업명(key)와 세부정보(value) 저장

            percentage_completed = (idx / total_urls) * 100
            print(f"[CareerScraper] 🔥 진척도 ------------------>> [{idx}/{total_urls}({percentage_completed:.2f}%)]")

        elapsed_time = time.time() - start_time
        print(f'[CareerScraper] 🔵 총 수집한 직업 데이터 개수 : {len(self.urls)} 개 | ⏱️ 소요 시간: {elapsed_time:.2f} 초')

    def _click_filter(self):
        """필터링 적용 - 리스트 보기, 30개씩 보기, 가나다 순 정렬"""
        ScrapingTools.wait_for_element(self.driver, "click", By.CSS_SELECTOR, '#ct > div.inner > div:nth-child(15) > div > span > a:nth-child(2)').click()  # '리스트 보기 아이콘' 버튼 클릭
        Select(self.driver.find_element(By.ID, "tagUl1")).select_by_value("30")  # '조회 개수'토글에서 "30개씩 보기" 옵션 선택
        Select(self.driver.find_element(By.ID, "tagUl2")).select_by_value("A")  # '조회 기준' 토글에서 "가나다순 보기" 옵션 선택
        ScrapingTools.wait_for_element(self.driver, "click", By.CSS_SELECTOR, '#ct > div.inner > div:nth-child(15) > div > div > a').click()  # "적용" 버튼 클릭
        time.sleep(2)

    def _get_contents_count(self):
        """전체 게시물 개수를 가져오기"""
        total_counts = int(self.driver.find_element(By.CSS_SELECTOR, "#totalCnt").text.strip())
        print(f"[CareerScraper] ⚪️ 전체 게시물 개수: {total_counts} 개")
        return 
    
    def _get_urls(self):
        """현재 페이지의 게시물 URL 수집"""
        rows = self.driver.find_elements(By.CSS_SELECTOR, '#searchTbody tr')
        for row in rows:
            link = row.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
            self.urls.append(link)

    def _click_nextpage(self, page_count):
        """다음 페이지로 이동"""
        try:
            self.driver.find_element(By.CSS_SELECTOR, f".pagination a[onclick*='fn_page({page_count})']").click()
            time.sleep(2)
            return True
        except Exception as e:
            print(f"[CareerScraper] 🔴 페이지 {page_count}로 이동하는 중 문제가 발생했습니다. : {e}")
            return False

    def _get_details(self):
        """직업 정보 세부 항목 수집"""
        job_details = {}
        job_details["직업명"] = self._get_element('.job_name')
        
        # 각 탭별 데이터 수집
        job_details["직업 개요"] = self._get_tab_data(1, [
            ("관련 직업명", '#tab1 .cont p.cont_txt'),
            ("관련 학과", '//dt[text()="관련학과"]/following-sibling::dd/a'),
            ("관련 자격증", '//dt[text()="관련자격"]/following-sibling::dd/a'),
            ("하는 일", '//h4[text()="하는일"]/following-sibling::ul/li'),
            ("필요한 핵심능력", '//h4[text()="핵심능력"]/following-sibling::span'),
            ("관련 적성", '//dt[text()="적성"]/following-sibling::dd'),
            ("관련 흥미", '//dt[text()="흥미"]/following-sibling::dd')
        ])

        job_details["직업 탐색 및 준비"] = self._get_tab_data(2, [
            ("관련 진로 탐색 활동", '//dt[text()="진로 탐색 활동"]/following-sibling::dd'),
            ("정규교육과정", '//dt[text()="정규교육과정"]/parent::dl/dd'),
            ("직업훈련", '//dt[text()="직업훈련"]/parent::dl/dd'),
            ("입직 및 취업방법", '//dt[text()="입직 및 취업방법"]/following-sibling::dd')
        ])

        job_details["직업현황 및 지표"] = self._get_tab_data(3, [
            ("직업전망", '//dt[text()="직업전망"]/parent::dl/dd/p'),
            ("평균연봉", '#tab3 > div:nth-child(1) > dl:nth-child(3) > dd > ul > li:nth-child(1) > p'),
            ("직업만족도", '#tab3 > div:nth-child(1) > dl:nth-child(3) > dd > ul > li:nth-child(2) > p'),
            ("학력분포", '#graph01_legend'),
            ("전공계열", '#graph02_legend')
        ])

        job_details["능력/지식/환경"] = self._get_tab_data(4, [
            ("업무수행능력", '//h4[text()="업무수행능력"]/following-sibling::table//tbody/tr'),
            ("지식중요도", '//h4[text()="지식중요도"]/following-sibling::table//tbody/tr'),
            ("업무환경", '//h4[text()="업무환경"]/following-sibling::table//tbody/tr')
        ])

        return job_details

    def _get_tab_data(self, tab_number, fields):
        """탭에서 데이터를 수집하는 함수"""
        if not self._click_tab(tab_number):
            print(f"[CareerScraper] 🔴 Tab{tab_number}이 존재하지 않음. 해당 데이터는 공백으로 채움")
            return {}

        tab_data = {}
        for field_name, selector in fields:
            tab_data[field_name] = self._get_element(selector)
        return tab_data

    def _get_element(self, selector, multiple=False):
        """요소를 찾지 못하면 빈 값 또는 리스트 반환"""
        try:
            if multiple:
                elements = self.driver.find_elements(By.CSS_SELECTOR if selector.startswith(".") else By.XPATH, selector)
                return [element.text.strip() for element in elements] if elements else []
            return self.driver.find_element(By.CSS_SELECTOR if selector.startswith(".") else By.XPATH, selector).text.strip()
        except Exception:
            return "" if not multiple else []

    def _click_tab(self, tab_number):
        """ 탭 클릭 시 탭이 없는 경우 False 반환 """
        try:
            tab_selector = f'li[data-tab="tab{tab_number}"] > a'
            self.driver.find_element(By.CSS_SELECTOR, tab_selector).click()
            time.sleep(1)  # 페이지 로딩 대기
            return True
        except Exception:
            return False  # 탭이 없을 경우 False 반환
        
    def save_results_markdown(self, filename):
        """직업 데이터에 맞게 포맷을 확장하여 마크다운으로 저장"""
        filename = filename+'.md'
        with open(filename, "w", encoding="utf-8") as f:
            for job_name, job_details in self.data.items():
                f.write(f"# {job_name}\n\n")
                
                # 각 탭의 데이터를 처리
                for tab_name, tab_details in job_details.items():
                    f.write(f"## {tab_name}\n\n")  # 탭 이름을 소제목으로 작성
                    
                    # tab에 대한 데이터가 없으면 = tab_details가 문자열이면 건너뛰기
                    if isinstance(tab_details, str):
                        f.write(f"{tab_details}\n\n")
                        continue
                    
                    # 각 탭 내부의 데이터 처리
                    for section_name, section_value in tab_details.items():
                        f.write(f"### {section_name}\n")  # 섹션 제목 작성
                        
                        # 섹션 값이 리스트일 경우
                        if isinstance(section_value, list):
                            for item in section_value:
                                if isinstance(item, dict):  # 리스트의 항목이 딕셔너리일 경우
                                    if section_name == "업무수행능력" or section_name == "지식중요도" or section_name == "업무환경":
                                        # 각각의 능력/지식/환경 항목에 대해 처리
                                        f.write(f"- {item.get('능력명', item.get('지식명', item.get('환경요소', '')))} "
                                                f"(중요도: {item.get('중요도', '')}) : {item.get('설명', '')}\n")
                                    else:
                                        f.write(f"- {', '.join(f'{k}: {v}' for k, v in item.items())}\n") # 일반적인 딕셔너리 항목 처리
                                else:
                                    f.write(f"- {item}\n") # 리스트의 항목이 문자열일 경우
                        else:
                            f.write(f"- {section_value}\n") # 섹션 값이 리스트가 아닌 경우
                    f.write("\n")  # 각 섹션 간 공백 추가
                f.write("\n---\n\n")  # 직업 정보 간에 구분선 추가
        print(f"[CareerScraper] 🟢 결과가 {filename} 파일에 저장되었습니다.")

# 5) 커리어넷 전공정보 데이터 수집을 위한 클래스 선언

- scraping_urls() : 전공정보 페이지에서 개별 전공정보가 담긴 게시물들의 URL 주소 수집
- scraping_data() : 전공정보 페이지에서 수집한 게시물 페이지 URL들을 순회하며 실제 데이터 수집
- click_tab() : 게시물 페이지에서 좌측 메뉴바(탭) 클릭
- 

In [9]:
class MajorScraper(BaseScraper):
    def scraping_urls(self):
        """ 학과 정보 페이지에서 개별 학과 정보가 담긴 페이지의 URL 주소 수집 """
        print("[MajorScraper] 🔵 학과 정보 데이터 수집 시작 (개별 학과 설명이 적힌 URL 수집)")
        start_time = time.time()
        self.driver.get(self.config["URL"]["careernet_majorinfo"])  # 커리어넷 학과정보 URL 접속

        self._click_filter()  # 일관성 있게 수집하기 위해 직업 정보 게시판 콘텐츠 정렬
        total_counts = self._get_contents_count("#disTemp0 > strong")  # 전체 게시물 개수 가져오기
        
        page_count = 1 # 페이지 번호 추적
        while len(self.urls) < total_counts: # 수집된 URL 개수가 '전체 게시물 개수'와 같을 때까지 수집
            self._get_urls()
            print(f"[MajorScraper] ⚪️ 페이지 {page_count} 스크래핑 완료. 수집한 누적 학과 데이터 개수: {len(self.urls)}")
            page_count += 1
            
            # 스크래핑 종료 기준
            if len(self.urls) >= total_counts:
                print("[MajorScraper] 🔵 모든 게시물을 다 스크래핑했습니다.")
                break
            if not self._click_nextpage(page_count):
                break
        
        elapsed_time = time.time() - start_time
        print(f'[MajorScraper] 🔵 총 수집한 학과 정보 URL 주소 : {len(self.urls)} 개 | ⏱️ 소요 시간: {elapsed_time:.2f} 초')
        
    def scraping_data(self):
        """ 학과 정보 URL 리스트에서 학과 정보 수집 """
        print(f"[MajorScraper] 🔵 URL 리스트로부터 데이터 수집 시작")
        start_time = time.time()

        total_urls = len(self.urls) # 전체 URL 개수
        for idx, url in enumerate(self.urls, start=1):
            print(f"[MajorScraper] 🔵 수집 타겟 URL 👉🏻 {url}")
            self.driver.get(url) # 학과 정보 페이지 접근
            major_details = self._get_details() # 학과 세부 정보 수집

            self.data[major_details["학과명"]] = major_details
            percentage_completed = (idx / total_urls) * 100
            print(f"[MajorScraper] 🔥 진척도 ------------------>> [{idx}/{total_urls} ({percentage_completed:.2f}%)]")

        elapsed_time = time.time() - start_time
        print(f'[MajorScraper] 🔵 총 수집한 학과 데이터 개수 : {len(self.urls)} 개 | ⏱️ 소요 시간: {elapsed_time:.2f} 초')

    def _click_filter(self):
        """필터링 적용 - 리스트 보기, 30개씩 보기, 가나다 순 정렬"""
        ScrapingTools.wait_for_element(self.driver, "click", By.CSS_SELECTOR, '#TabMenu3 > a.icon_list').click() # '리스트 보기 아이콘' 버튼 클릭
        Select(self.driver.find_element(By.ID, "tagUl1")).select_by_value("30")                                  # '조회 개수'토글에서 "30개씩 보기" 옵션 선택
        Select(self.driver.find_element(By.ID, "tagUl2")).select_by_value("A")                                   # '조회 기준' 토글에서 "가나다순 보기" 옵션 선택
        ScrapingTools.wait_for_element(self.driver, "click", By.CSS_SELECTOR, '#frm > div:nth-child(12) > div > div > a').click() # "적용" 버튼 클릭
        time.sleep(2)

    def _get_contents_count(self, selector):
        """전체 게시물 개수를 가져오기"""
        total_counts = int(self.driver.find_element(By.CSS_SELECTOR, selector).text.strip())
        print(f"[MajorScraper] ⚪️ 전체 게시물 개수: {total_counts} 개")
        return total_counts
    
    def _get_urls(self):
        """현재 페이지의 게시물 URL 수집"""
        # 현재 페이지의 '개별 직업 정보 페이지 URL' 수집
        rows = self.driver.find_elements(By.CSS_SELECTOR, '#view2 > div.board_list > table > tbody > tr')
        for row in rows:
            link = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2) > a').get_attribute('href')
            self.urls.append(link)  # urls 리스트에 저장

    def _click_nextpage(self, page_count):
        """다음 페이지로 이동"""
        try:
            element = self.driver.find_element(By.CSS_SELECTOR, f".pagination a[onclick*='fn_list2({page_count})']")
            self.driver.execute_script("arguments[0].click();", element)
            time.sleep(2)
            return True
        except Exception as e:
            print(f"[MajorScraper] 🔴 페이지 {page_count}로 이동하는 중 문제가 발생했습니다: {e}")
            return False

    def _get_details(self):
        """학과 세부 항목 수집"""
        major_details = {}
        major_details["학과명"] = self._get_element('.word_tit h2', multiple=False)

        # 각 탭별 데이터 수집
        major_details["학과 개요"] = self._get_tab_data(1, [
            ("취업률", '.class1 strong'),
            ("첫 직장 임금", '.class2 span'),
            ("학과개요", '#tab1 ul.word_ul:nth-of-type(1) li'),
            ("학과특성", '#tab1 > ul:nth-of-type(2) > li'),
            ("흥미와 적성", '#tab1 > ul:nth-child(7) > li'),
            ("관련 고교 교과목", '//*[@id="tab1"]/ul[4]/li'),
            ("대학 주요 교과목", '//*[@id="tab1"]/ul[6]/li'),
            ("졸업 후 진출 분야", '//*[@id="tab1"]/ul[9]/li'),
            ("진로 탐색 활동", '#tab1 > ul:nth-child(11) > li'),
            ("관련 자격", '#tab1 > ul:nth-child(15) > li'),
            ("관련 직업", '#tab1 > ul:nth-child(17) > li'),
            ("세부관련학과", '#tab1 > ul:nth-child(23) > li')
        ])

        major_details["개설대학"] = self._get_details_univinfo()
        major_details["인터뷰"] = self._get_details_interview()
        return major_details

    def _get_details_univinfo(self):
        """개설대학 데이터를 수집하는 함수"""
        if not self._click_tab(2):
            print("[MajorScraper] 🔴 Tab2(개설대학)가 존재하지 않음. 해당 데이터는 공백으로 채움")
            return []

        universities = []
        try:
            rows = self.driver.find_elements(By.CSS_SELECTOR, 'table.table_job_st1 tbody tr')
            for row in rows:
                region = row.find_elements(By.TAG_NAME, 'td')[0].text.strip()
                university = row.find_elements(By.TAG_NAME, 'td')[1].text.strip()
                major = row.find_elements(By.TAG_NAME, 'td')[2].text.strip()
                universities.append({
                    "지역": region,
                    "대학명": university,
                    "학과명": major
                })
        except Exception as e:
            print(f"[MajorScraper] 🔴 개설대학 수집 중 오류 발생: {e}")
            return []
        
        return universities

    def _get_details_interview(self):
        """인터뷰 데이터를 수집하는 함수"""
        if not self._click_tab(4):
            print("[MajorScraper] 🔴 Tab4(인터뷰)가 존재하지 않음. 해당 데이터는 공백으로 채움")
            return {"학과명": "", "교수명": "", "소속": "", "질문답변": []}

        interview_details = {}
        try:
            interview_details["학과명"] = self._get_element('.intvw_head h2', multiple=False)
            interview_details["교수명"] = self._get_element('.intvw_head p', multiple=False)
            interview_details["소속"] = self._get_element('.intvw_head h3', multiple=False)

            # 질문과 답변 수집
            interview_details["질문답변"] = []
            questions = self.driver.find_elements(By.CSS_SELECTOR, '.question .bubble')
            answers = self.driver.find_elements(By.CSS_SELECTOR, '.answer .bubble')
            for question, answer in zip(questions, answers):
                interview_details["질문답변"].append({
                    "질문": question.text.strip(),
                    "답변": answer.text.strip()
                })
        except Exception as e:
            print(f"[MajorScraper] 🔴 인터뷰 수집 중 오류 발생: {e}")
            return {"학과명": "", "교수명": "", "소속": "", "질문답변": []}

        return interview_details

    def _get_tab_data(self, tab_number, fields):
        """탭에서 데이터를 수집하는 함수"""
        if not self._click_tab(tab_number):
            print(f"[MajorScraper] 🔴 Tab{tab_number}이 존재하지 않음. 해당 데이터는 공백으로 채움")
            return {}

        tab_data = {}
        for field_name, selector in fields:
            tab_data[field_name] = self._get_element(selector)
        return tab_data
         
    def _click_tab(self, tab_number):
        """ 탭 클릭 시 탭이 존재하면 True 반환, 없으면 False 반환 """
        try:
            if tab_number == 1:
                tab_selector = "#JQTabMenu > ul > li.ui-state-default.ui-corner-top.ui-tabs-selected.ui-state-active > a"
            else:
                tab_selector = f"#JQTabMenu > ul > li:nth-child({tab_number}) > a"
            self.driver.find_element(By.CSS_SELECTOR, tab_selector).click()
            time.sleep(1)
            return True
        except Exception as e:
            print(f"[MajorScraper] 🔴 탭 {tab_number}을 클릭하는 중 오류 발생: {e}")
            return False 
        
    def _get_element(self, selector, multiple=True):
        """ 
        - Xpath, CSS Selector 자동 탐지함
        - multiple = True -> 여러개의 요소를 리스트에 수집
        - multiple = False -> 한개의 요소를 문자열로 수집
        """
        try:
            if selector.startswith("//"): # Xpath, CSS Selector 구분
                method = By.XPATH
            else:
                method = By.CSS_SELECTOR
            
            if multiple:
                elements = self.driver.find_elements(method, selector)
                return [element.text.strip() for element in elements] if elements else []
            else: 
                element = self.driver.find_element(method, selector)
                return element.text.strip() if element else ""
        except Exception as e:
            print(f"[MajorScraper] 🔴 요소를 찾을 수 없습니다. 오류: {str(e)} -> 해당 값을 빈 값으로 채웁니다.")
            return [] if multiple else ""
        
    def save_results_markdown(self, filename):
        """ 데이터를 마크다운 형식으로 변환하여 파일로 저장 """
        filename = filename+'.md'
        try:
            with open(filename, 'w', encoding='utf-8') as file:
                for department, details in self.data.items():
                    # 학과 제목
                    file.write(f"# {department}\n\n")

                    # 학과 개요
                    overview = details.get("학과 개요", {})
                    if overview:
                        file.write("## 전공 개요\n\n")
                        file.write(f"- 취업률 : {overview.get('취업률', '')}\n")
                        file.write(f"- 평균 첫 임금 : {overview.get('첫 직장 임금', '')}\n\n")

                        # 학과개요
                        file.write("### 전공 설명\n\n")
                        file.write(f"{overview.get('학과개요', '')}\n\n")

                        # 학과특성
                        file.write("### 전공 특성\n\n")
                        file.write(f"{overview.get('학과특성', '')}\n\n")

                        # 흥미와 적성
                        file.write("### 전공 관련 흥미와 적성\n\n")
                        file.write(f"{overview.get('흥미와 적성', '')}\n\n")

                        # 관련 고교 교과목
                        file.write("### 관련 고교 교과목\n\n")
                        highschool_subjects = overview.get("관련 고교 교과목", {})
                        for key, value in highschool_subjects.items():
                            file.write(f"- {key} : {value}\n")
                        file.write("\n")

                        # 대학 주요 교과목
                        file.write("### 대학 주요 교과목\n")
                        major_subjects = overview.get("대학 주요 교과목", {})
                        for subject, description in major_subjects.items():
                            file.write(f"- {subject} :  \n{description}\n\n")

                        # 졸업 후 진출 분야
                        file.write("### 졸업 후 진출 분야\n\n")
                        career_paths = overview.get("졸업 후 진출 분야", {})
                        for field, jobs in career_paths.items():
                            file.write(f"- {field} :  \n{jobs}\n\n")

                        # 진로 탐색 활동
                        file.write("### 관련 진로 탐색 활동\n\n")
                        file.write(f"{overview.get('진로 탐색 활동', '')}\n\n")

                        # 관련 자격증
                        file.write("### 관련 자격증\n\n")
                        for cert in overview.get("관련 자격", []):
                            file.write(f"- {cert}\n")
                        file.write("\n")

                        # 관련 직업
                        file.write("### 관련 직업\n\n")
                        for job in overview.get("관련 직업", []):
                            file.write(f"- {job}\n")
                        file.write("\n")

                        # 세부관련학과
                        file.write("### 세부관련학과\n\n")
                        for related_major in overview.get("세부관련학과", []):
                            file.write(f"- {related_major}\n")
                        file.write("\n")

                    # 개설대학
                    universities = details.get("개설대학", [])
                    if universities:
                        file.write("## 개설대학\n\n")
                        for uni in universities:
                            region = uni.get("지역", "")
                            university_name = uni.get("대학명", "")
                            major_name = uni.get("학과명", "")
                            file.write(f"- {university_name}(지역: {region}) {major_name} \n")
                        file.write("\n")

                    # 인터뷰
                    interview = details.get("인터뷰", {})
                    if interview:
                        file.write("## 인터뷰\n\n")
                        file.write(f"- 학과명: {interview.get('학과명', '')}\n")
                        file.write(f"- 교수명: {interview.get('교수명', '')}\n")
                        file.write(f"- 소속: {interview.get('소속', '')}\n\n")

                        # 질문과 답변
                        file.write("### 내용:\n\n")
                        for qna in interview.get("질문답변", []):
                            question = qna.get("질문", "")
                            answer = qna.get("답변", "")
                            file.write(f"> 질문 : {question}\n\n")
                            file.write(f"{answer}\n\n")

            print(f"[MajorScraper] 🟢 결과가 {filename} 파일에 성공적으로 저장되었습니다.")
        
        except Exception as e:
            print(f"[MajorScraper] 🔴 파일 저장 중 오류 발생: {e}")

# 6) 커리어넷 대학정보 데이터 수집을 위한 클래스 선언

- scraping_urls() : 대학정보 페이지에서 개별 대학정보가 담긴 게시물들의 URL 주소 수집
- scraping_data() : 대학정보 페이지에서 수집한 게시물 페이지 URL들을 순회하며 실제 데이터 수집
- click_tab() : 게시물 페이지에서 좌측 메뉴바(탭) 클릭
- 

In [8]:
class UnivScraper(BaseScraper):  
    def scraping_urls(self):
        return super().scraping_urls() 
    
    def scraping_data(self):
        """ 대학 정보 페이지에서 개별 대학 정보 수집 """
        print("[UnivScraper] 🔵 대학 정보 데이터 수집 시작")
        start_time = time.time()
        self.driver.get(self.config["URL"]["careernet_univinfo"])  # 커리어넷 대학정보
        
        # 조회 개수 설정 (30개씩 보기)
        self._click_filter()
        
        total_counts = self._get_contents_count(".result_list_info strong")  # 전체 게시물 개수 가져오기
        page_count = 1  # 페이지 번호 추적

        while len(self.data) < total_counts:
            self._get_details()  # 데이터 수집
            print(f"[UnivScraper] ⚪️ 페이지 {page_count} 스크래핑 완료. 수집한 누적 데이터 개수: {len(self.data)}")
            page_count += 1

            # 스크래핑 종료 기준
            if len(self.data) >= total_counts:
                print("[UnivScraper] 🔵 모든 게시물을 다 스크래핑했습니다.")
                break

            if not self._click_nextpage(page_count):
                break

        elapsed_time = time.time() - start_time
        print(f'[UnivScraper] 🔵 총 수집한 대학 정보: {len(self.data)} 개 | ⏱️ 소요 시간: {elapsed_time:.2f} 초')

    def _click_filter(self):
        """ 일관성 있게 수집하기 위해 직업 정보 게시판 콘텐츠 정렬"""
        dropdown_count = Select(self.driver.find_element(By.ID, "tagUl1")) # '조회 개수' 토글 클릭
        dropdown_count.select_by_value("30") # '조회 개수'토글에서 "30개씩 보기" 옵션 선택
        ScrapingTools.wait_for_element(self.driver, "click", By.CSS_SELECTOR, '#frm > div:nth-child(8) > div > div > a').click() # "적용" 버튼 클릭
        time.sleep(2)

    def _get_contents_count(self, selector):
        """전체 게시물 개수를 가져오기"""
        total_counts = int(self.driver.find_element(By.CSS_SELECTOR, selector).text.strip())
        print(f"[UnivScraper] ⚪️ 전체 게시물 개수: {total_counts} 개")
        return total_counts

    def _get_details(self):
        """현재 페이지의 대학 정보 수집"""
        rows = self.driver.find_elements(By.CSS_SELECTOR, "tbody tr")
        for row in rows:
            try:
                name, url = self._get_details_name_url(row)
                university_type = row.find_elements(By.CSS_SELECTOR, "td.tc")[1].text.strip()  # 학교 종류
                university_category = row.find_elements(By.CSS_SELECTOR, "td.tc")[2].text.strip()  # 학교 유형
                ownership = row.find_elements(By.CSS_SELECTOR, "td.tc")[3].text.strip()  # 설립
                location = row.find_elements(By.CSS_SELECTOR, "td.tc")[4].text.strip()  # 지역

                # 대학 정보를 딕셔너리로 저장
                self.data[name] = {
                    "URL": url,
                    "학교종류": university_type,
                    "학교유형": university_category,
                    "설립": ownership,
                    "지역": location,
                }
            except Exception as e:
                print(f"[UnivScraper] 🔴 데이터 수집 중 오류 발생: {e}")

    def _get_details_name_url(self, row):
        """대학명과 URL 수집"""
        name_element = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)")
        if name_element.find_elements(By.TAG_NAME, "a"):
            name = name_element.find_element(By.TAG_NAME, "a").text.strip()
            url = name_element.find_element(By.TAG_NAME, "a").get_attribute("href")
        else:
            name = name_element.text.strip() if name_element.text.strip() else "N/A"
            url = "N/A"
        return name, url

    def _click_nextpage(self, page_count):
        """다음 페이지로 이동"""
        try:
            self.driver.find_element(By.CSS_SELECTOR, f".pagination a[onclick*='fn_list({page_count})']").click()
            time.sleep(2)
            return True
        except Exception as e:
            print(f"[UnivScraper] 🔴 페이지 {page_count}로 이동하는 중 문제가 발생했습니다: {e}")
            return False
    
    def run_scraping(self, filename):
        """URL 수집, 데이터 수집, 결과 저장을 일괄 실행"""
        print(f"[BaseScraper] 🟢 스크래핑 작업 시작")
        self.scraping_data() # 데이터 수집        
        self.save_results_markdown(filename=filename) # Step 3: 결과 저장
        print(f"[BaseScraper] 🟢 스크래핑 작업 완료")
        
    def save_results_markdown(self, filename):
        """ 데이터를 마크다운 형식으로 변환하여 파일로 저장 """
        filename = filename + '.md'
        try:
            with open(filename, "w", encoding="utf-8") as f:
                # 데이터를 순차적으로 Markdown 형식으로 저장
                for univ_name, univ_details in self.data.items():
                    f.write(f"# {univ_name}\n\n")  # 대학명은 헤더로 표시

                    # 대학 세부 정보를 하나씩 작성
                    for key, value in univ_details.items():
                        f.write(f"- {key}: {value}\n")
                    f.write("\n---\n\n")  # 각 대학 정보를 구분하는 구분선 추가
            print(f"[UnivScraper] 🟢 결과가 {filename} 파일에 성공적으로 저장되었습니다.")
        except Exception as e:
            print(f"[UnivScraper] 🔴 파일 저장 중 오류 발생: {e}")
        

# 7) 스크래핑 실행

---

In [14]:
ROOT_PATH = "./result"

print("====================== [ Career ] ======================")
career_scraper = CareerScraper(config)
career_scraper.run_scraping(f"{ROOT_PATH}/CareerNet_CareerInfo") # 직업 데이터

====================== [ Career ] ======================
[ScrapingTools] 🟡 크롬 드라이버 객체 생성 완료
[BaseScraper] 🟢 스크래핑 작업 시작
[CareerScraper] 🔵 직업 정보 데이터 수집 시작 (개별 직업 설명이 적힌 URL 수집)
[CareerScraper] ⚪️ 전체 게시물 개수: 540 개
[CareerScraper] ⚪️ 페이지 1 스크래핑 완료. 수집한 누적 직업 데이터 개수: 30
[CareerScraper] ⚪️ 페이지 2 스크래핑 완료. 수집한 누적 직업 데이터 개수: 60
[CareerScraper] ⚪️ 페이지 3 스크래핑 완료. 수집한 누적 직업 데이터 개수: 90
[CareerScraper] ⚪️ 페이지 4 스크래핑 완료. 수집한 누적 직업 데이터 개수: 120
[CareerScraper] ⚪️ 페이지 5 스크래핑 완료. 수집한 누적 직업 데이터 개수: 150
[CareerScraper] ⚪️ 페이지 6 스크래핑 완료. 수집한 누적 직업 데이터 개수: 180
[CareerScraper] ⚪️ 페이지 7 스크래핑 완료. 수집한 누적 직업 데이터 개수: 210
[CareerScraper] ⚪️ 페이지 8 스크래핑 완료. 수집한 누적 직업 데이터 개수: 240
[CareerScraper] ⚪️ 페이지 9 스크래핑 완료. 수집한 누적 직업 데이터 개수: 270
[CareerScraper] ⚪️ 페이지 10 스크래핑 완료. 수집한 누적 직업 데이터 개수: 300
[CareerScraper] ⚪️ 페이지 11 스크래핑 완료. 수집한 누적 직업 데이터 개수: 330
[CareerScraper] ⚪️ 페이지 12 스크래핑 완료. 수집한 누적 직업 데이터 개수: 360
[CareerScraper] ⚪️ 페이지 13 스크래핑 완료. 수집한 누적 직업 데이터 개수: 390
[CareerScraper] ⚪️ 페이지 14 스크래핑 완료. 수집한 누적 직업 데이터 개수: 420
[Ca

In [11]:
print("====================== [ Major ] ======================")
ROOT_PATH = "./result"
major_scraper = MajorScraper(config)
major_scraper.run_scraping(f"{ROOT_PATH}/CareerNet_MajorInfo") # 전공 데이터(학과) 

====================== [ Major ] ======================
[ScrapingTools] 🟡 크롬 드라이버 객체 생성 완료
[BaseScraper] 🟢 스크래핑 작업 시작
[MajorScraper] 🔵 학과 정보 데이터 수집 시작 (개별 학과 설명이 적힌 URL 수집)
[MajorScraper] ⚪️ 전체 게시물 개수: 501 개
[MajorScraper] ⚪️ 페이지 1 스크래핑 완료. 수집한 누적 직업 데이터 개수: 30
[MajorScraper] ⚪️ 페이지 2 스크래핑 완료. 수집한 누적 직업 데이터 개수: 60
[MajorScraper] ⚪️ 페이지 3 스크래핑 완료. 수집한 누적 직업 데이터 개수: 90
[MajorScraper] ⚪️ 페이지 4 스크래핑 완료. 수집한 누적 직업 데이터 개수: 120
[MajorScraper] ⚪️ 페이지 5 스크래핑 완료. 수집한 누적 직업 데이터 개수: 150
[MajorScraper] ⚪️ 페이지 6 스크래핑 완료. 수집한 누적 직업 데이터 개수: 180
[MajorScraper] ⚪️ 페이지 7 스크래핑 완료. 수집한 누적 직업 데이터 개수: 210
[MajorScraper] ⚪️ 페이지 8 스크래핑 완료. 수집한 누적 직업 데이터 개수: 240
[MajorScraper] ⚪️ 페이지 9 스크래핑 완료. 수집한 누적 직업 데이터 개수: 270
[MajorScraper] ⚪️ 페이지 10 스크래핑 완료. 수집한 누적 직업 데이터 개수: 300
[MajorScraper] ⚪️ 페이지 11 스크래핑 완료. 수집한 누적 직업 데이터 개수: 330
[MajorScraper] ⚪️ 페이지 12 스크래핑 완료. 수집한 누적 직업 데이터 개수: 360
[MajorScraper] ⚪️ 페이지 13 스크래핑 완료. 수집한 누적 직업 데이터 개수: 390
[MajorScraper] ⚪️ 페이지 14 스크래핑 완료. 수집한 누적 직업 데이터 개수: 420
[MajorScraper] ⚪️ 페이

In [13]:
print("====================== [ Univ ] ======================")
ROOT_PATH = "./result"
univ_scraper = UnivScraper(config)
univ_scraper.run_scraping(f"{ROOT_PATH}/CareerNet_UnivInfo") # 대학 데이터

====================== [ Univ ] ======================
[ScrapingTools] 🟡 크롬 드라이버 객체 생성 완료
[BaseScraper] 🟢 스크래핑 작업 시작
[UnivScraper] 🔵 대학 정보 데이터 수집 시작
[UnivScraper] ⚪️ 전체 게시물 개수: 476 개
[UnivScraper] ⚪️ 페이지 1 스크래핑 완료. 수집한 누적 직업 데이터 개수: 30
[UnivScraper] ⚪️ 페이지 2 스크래핑 완료. 수집한 누적 직업 데이터 개수: 60
[UnivScraper] ⚪️ 페이지 3 스크래핑 완료. 수집한 누적 직업 데이터 개수: 90
[UnivScraper] ⚪️ 페이지 4 스크래핑 완료. 수집한 누적 직업 데이터 개수: 120
[UnivScraper] ⚪️ 페이지 5 스크래핑 완료. 수집한 누적 직업 데이터 개수: 150
[UnivScraper] ⚪️ 페이지 6 스크래핑 완료. 수집한 누적 직업 데이터 개수: 180
[UnivScraper] ⚪️ 페이지 7 스크래핑 완료. 수집한 누적 직업 데이터 개수: 210
[UnivScraper] ⚪️ 페이지 8 스크래핑 완료. 수집한 누적 직업 데이터 개수: 240
[UnivScraper] ⚪️ 페이지 9 스크래핑 완료. 수집한 누적 직업 데이터 개수: 270
[UnivScraper] ⚪️ 페이지 10 스크래핑 완료. 수집한 누적 직업 데이터 개수: 300
[UnivScraper] ⚪️ 페이지 11 스크래핑 완료. 수집한 누적 직업 데이터 개수: 330
[UnivScraper] ⚪️ 페이지 12 스크래핑 완료. 수집한 누적 직업 데이터 개수: 360
[UnivScraper] ⚪️ 페이지 13 스크래핑 완료. 수집한 누적 직업 데이터 개수: 390
[UnivScraper] ⚪️ 페이지 14 스크래핑 완료. 수집한 누적 직업 데이터 개수: 420
[UnivScraper] ⚪️ 페이지 15 스크래핑 완료. 수집한 누적 직업 데이터 개수: 450
[Uni

---

# 8) 수집한 데이터 개수 확인

In [15]:
import re

def count_titles(filename):
    """ 마크다운 파일에서 H1(#) 제목의 개수 파악 """
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()

        # 정규 표현식을 사용하여 '# ' 패턴만 추출 (즉, H1 제목만)
        h1_count = len(re.findall(r'^# ', content, re.MULTILINE))

        print(f"파일 '{filename}'의 H1(#) 제목 개수 : {h1_count}")
        return h1_count
    
    except Exception as e:
        print(f"파일을 불러오는 중 오류 발생: {e}")
        return None


careerinfo = './result/CareerNet_CareerInfo.md'
count_titles(careerinfo)

majorinfo = './result/CareerNet_MajorInfo.md'
count_titles(majorinfo)

univinfo = './result/CareerNet_UnivInfo.md'
count_titles(univinfo)

파일 './result/CareerNet_CareerInfo.md'의 H1(#) 제목 개수 : 540
파일 './result/CareerNet_MajorInfo.md'의 H1(#) 제목 개수 : 500
파일 './result/CareerNet_UnivInfo.md'의 H1(#) 제목 개수 : 476


476